### 針對mysql(db名test、table名user)的資料做新增、刪除、查詢等動作的設計
### 先讓自己連上資料庫(phpmyadmin)

In [ ]:
!pip install flask # WEB框架
!pip install flask-restful # 可把URL名稱寫得簡略，並將動作藏在HTTP的method裡面
!pip install pymysql

In [ ]:
from flask import jsonify
from flask_restful import Resource, reqparse
import pymysql

class Users(Resource):
    def db_init(self):
        db = pymysql.connect(
          host='127.0.0.1',
          user='root',
          password='root',
          port=3306,
          db='test'
      )
        cursor = db.cursor(pymysql.cursors.DictCursor)
        return db, cursor


    def get(self):
        db, cursor = self.db_init()
        parser = reqparse.RequestParser()
        parser.add_argument('name')
        args = parser.parse_args()
        name = args.get("name")
        if name is not None:
            sql = f"SELECT * FROM test.user WHERE name = '{name}';"
        else:
            sql = 'SELECT * FROM test.user;'

        cursor.execute(sql)
        users = cursor.fetchall()
        db.close()
        return jsonify(users)
        # 在postman用GET查詢http://127.0.0.1:10009/users會出現users的表格內容

    def post(self):      
        db, cursor = self.db_init()
        parser = reqparse.RequestParser()
        parser.add_argument('name', required=True)
        parser.add_argument('gender', required=True)
        parser.add_argument('birth')
        parser.add_argument('note')
        args = parser.parse_args()
        user = {
          'name': args['name'],
          'gender': args['gender'],
          'birth': args.get('birth') or '1900-01-01',
          'note': args.get('note')
        }
        sql = """
        INSERT INTO `test`.`user` (`name`,`gender`,`birth`,`note`)
        VALUES ('{}','{}','{}','{}');

        """.format(user['name'], user['gender'], user['birth'], user['note'])
        result = cursor.execute(sql)
        db.commit()
        db.close()
        response = {"message": "success"}
        return jsonify(response)
        # 在postman用POST查詢http://127.0.0.1:10009/users並在"body"-"form data"輸入post的資料就可以新增資料

class User(Resource):
    def db_init(self):
        db = pymysql.connect(
          host='127.0.0.1',
          user='root',
          password='root',
          port=3306,
          db='test'
        )
        cursor = db.cursor(pymysql.cursors.DictCursor)
        return db, cursor

    # Get single by id
    def get(self, id):
        db, cursor = self.db_init()
        print(db, cursor)
        sql = f"SELECT * FROM test.user WHERE id = '{id}';"
        cursor.execute(sql)
        users = cursor.fetchall()
        db.close()
        return jsonify(users)
        # 在postman用GET查詢http://127.0.0.1:10009/user/1就可以查詢到id=1的資料

    def patch(self, id):
        db, cursor = self.db_init()
        parser = reqparse.RequestParser()
        parser.add_argument('name')
        parser.add_argument('gender')
        parser.add_argument('birth')
        parser.add_argument('note')
        args = parser.parse_args()
        user = {
          'name': args.get('name'),
          'gender': args.get('gender'),
          'birth': args.get('birth'),
          'note': args.get('note')
        }
        query = []
        for key, value in user.items():
            if value is not None:
                query.append(f"{key} = '{value}'")
        query = ",".join(query)
        sql = """
            UPDATE test.user
            SET {}
            WHERE id = {};
        """.format(query, id)
        print(sql)
        result = cursor.execute(sql)
        db.commit()
        db.close()
        response = {"message": "success"}   
        return jsonify(response)
        # 在postman用PATCH查詢http://127.0.0.1:10009/user/1並在"body"-"form data"輸入修改後的資料就可以修改id=1的資料
  
    def delete(self, id):
        db, cursor = self.db_init()
        sql = f'DELETE FROM `test`.`user` WHERE id = {id};'
        result = cursor.execute(sql)
        db.commit()
        db.close()
        response = {"message": "success"}
        return jsonify(response)
        # 在postman用DELETE查詢http://127.0.0.1:10009/user/1就可以刪除id=1的資料